## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weatherpy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,60.96,94,20,5.75,few clouds
1,1,San Patricio,US,28.0170,-97.5169,84.09,60,5,10.16,clear sky
2,2,Kapaa,US,22.0752,-159.3190,73.72,97,75,16.11,broken clouds
3,3,Almansa,ES,38.8692,-1.0971,80.69,28,100,10.78,overcast clouds
4,4,Bathsheba,BB,13.2167,-59.5167,83.03,73,10,19.06,clear sky
5,5,Dudinka,RU,69.4058,86.1778,35.13,100,100,5.97,overcast clouds
6,6,Laramie,US,41.3114,-105.5911,59.54,44,0,12.66,clear sky
7,7,Xichang,CN,27.8964,102.2634,52.52,90,100,1.14,overcast clouds
8,8,Tasiilaq,GL,65.6145,-37.6368,32.16,97,100,14.32,snow
9,9,Victoria,HK,22.2855,114.1577,71.65,94,100,5.01,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,60.96,94,20,5.75,few clouds
1,1,San Patricio,US,28.0170,-97.5169,84.09,60,5,10.16,clear sky
2,2,Kapaa,US,22.0752,-159.3190,73.72,97,75,16.11,broken clouds
3,3,Almansa,ES,38.8692,-1.0971,80.69,28,100,10.78,overcast clouds
4,4,Bathsheba,BB,13.2167,-59.5167,83.03,73,10,19.06,clear sky
9,9,Victoria,HK,22.2855,114.1577,71.65,94,100,5.01,overcast clouds
15,15,Rikitea,PF,-23.1203,-134.9692,76.80,71,100,19.33,overcast clouds
16,16,Albany,US,42.6001,-73.9662,71.64,80,95,3.62,overcast clouds
18,18,Annau,TM,37.8876,58.5160,79.36,33,0,5.66,clear sky
20,20,Ponta Do Sol,PT,32.6667,-17.1000,78.71,66,54,8.81,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                375
City                   375
Country                371
Lat                    375
Lng                    375
Max Temp               375
Humidity               375
Cloudiness             375
Wind Speed             375
Current Description    375
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
#clean_df.isnull().sum()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,60.96,few clouds,-21.2000,-175.2000,
1,San Patricio,US,84.09,clear sky,28.0170,-97.5169,
2,Kapaa,US,73.72,broken clouds,22.0752,-159.3190,
3,Almansa,ES,80.69,overcast clouds,38.8692,-1.0971,
4,Bathsheba,BB,83.03,clear sky,13.2167,-59.5167,
9,Victoria,HK,71.65,overcast clouds,22.2855,114.1577,
15,Rikitea,PF,76.80,overcast clouds,-23.1203,-134.9692,
16,Albany,US,71.64,overcast clouds,42.6001,-73.9662,
18,Annau,TM,79.36,clear sky,37.8876,58.5160,
20,Ponta Do Sol,PT,78.71,broken clouds,32.6667,-17.1000,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [16]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df.head(10)
#clean_hotel_df = hotel_df.dropna()
#clean_hotel_df.isnull().sum()

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))